In [ ]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")
include("Draw.jl")
include("Escape.jl")
include("cross_escape.jl")

In [ ]:

chrms = Chromosome[]
best_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
worst_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
crossover_functions = [5,7,8]
instance = :eil51
K = 3
T = Read_TSPLIB_instance(instance, 1)
tsp = readTSPLIB(instance)
allNodes = tsp.nodes
num_of_nodes = size(allNodes)[1] - 1
depot = allNodes[1, :]
Customers = allNodes[2:num_of_nodes+1, :];
n = size(T)[1]-2
n_nodes = n
demands = ones(Int, n)
W = 150
h = 0.3
popsize = (20,40)
k_tournament = 2
num_iter = 2000
Mutation_Chance = 0.0

num_runs = 10
num_nei = 2
avg = 0.0
best = Inf
worst = 0.0
t1 = time() 

for i=1:num_runs
    P, roullet = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, 
        k_tournament, num_iter, Mutation_Chance, num_nei, crossover_functions);
#             roullet_ = roullet_ + roullet
    push!(chrms, P[1])
    avg += P[1].fitness
    if P[1].fitness < best
        best = P[1].fitness
        best_chrm = P[1]
    end
    if P[1].fitness > worst
        worst = P[1].fitness
        worst_chrm = P[1]
    end
end
t2 = time()
println("Results for ", instance, " ,m=", K)
println("Best: ", round(best, digits = 1), "  Average: ", round(avg/num_runs, digits = 1), 
    "  Worst: ", round(worst, digits = 1), " , run time= ", round((t2-t1)/num_runs, digits=0))


In [261]:
chrm = deepcopy(worst_chrm)

Chromosome([21, 1, 48, 38, 29, 33, 20, 28, 19, 34  …  18, 41, 43, 14, 44, 32, 9, 8, 49, 15], 171.29909226766915, 142.33763671650743, Tour[Tour([21, 1, 48, 38, 29, 33, 20, 28, 19, 34, 35, 2, 27, 30, 25, 7], 170.79170443197282), Tour([47, 6, 22, 42, 23, 5, 26, 50, 45, 11, 46, 17, 3, 16, 36, 4, 37, 10, 31], 167.43610928278088), Tour([13, 24, 12, 40, 39, 18, 41, 43, 14, 44, 32, 9, 8, 49, 15], 171.29909226766915)])

In [181]:
tour1 = copy(chrm.tours[1].Sequence)
tour2 = copy(chrm.tours[2].Sequence);
cost1 = chrm.tours[1].cost
cost2 = chrm.tours[2].cost
length(tour1)

26

In [217]:
k11 = 3
k12 = 6
k21 = 7
k22 = 13

for k13= k12+2:length(tour1) - (k22-k21)
    for k23=k21+2:length(tour2) - (k12-k11) 
        k11 = 3
        k12 = 6
        k21 = 7
        k22 = 13
        tour1 = copy(chrm.tours[1].Sequence)
        tour2 = copy(chrm.tours[2].Sequence);
        cost1 = chrm.tours[1].cost
        cost2 = chrm.tours[2].cost
        n_nodes = n

        c1 = sum(T[tour1[i]+1, tour1[i+1]+1] for i=k11:k12-1)
        c2 = sum(T[tour2[i]+1, tour2[i+1]+1] for i=k21:k22-1)
        t1 = copy(tour1)
        t2 = copy(tour2)
        pushfirst!(t1, 0)
        push!(t1, n_nodes+1)
        pushfirst!(t2, 0)
        push!(t2, n_nodes+1)
        k11 += 1
        k12 += 1
        k21 += 1
        k22 += 1
        k13 += 1
        k23 += 1
        if k13 == k11
            cost11 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t2[k21]+1] + c2 + T[t2[k22]+1, t1[k12+1]+1]
            cost12 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t2[k22]+1] + c2 + T[t2[k21]+1, t1[k12+1]+1]
        else
            cost11 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t1[k12+1]+1] + T[t1[k13-1]+1, t2[k21]+1] + c2 + T[t2[k22]+1, t1[k13]+1] - T[t1[k13-1]+1, t1[k13]+1]
            cost12 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t1[k12+1]+1] + T[t1[k13-1]+1, t2[k22]+1] + c2 + T[t2[k21]+1, t1[k13]+1] - T[t1[k13-1]+1, t1[k13]+1]
        end

        # print("cost1 - T[$(t1[k11-1]), $(t1[k11])] - c1 - T[$(t1[k12]), $(t1[k12+1])] + T[$(t1[k11-1]), $(t1[k12+1])] + T[$(t1[k13-1]), $(t2[k21])] + c2 + T[$(t2[k22]), $(t1[k13])] - T[$(t1[k13-1]), $(t1[k13])]")
        if k23 == k21
            cost21 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t1[k11]+1] + c1 + T[t1[k12]+1, t2[k22+1]+1]
            cost22 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t1[k12]+1] + c1 + T[t1[k11]+1, t2[k22+1]+1]
        else
            cost21 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t2[k22+1]+1] + T[t2[k23-1]+1, t1[k11]+1] + c1 + T[t1[k12]+1, t2[k23]+1] - T[t2[k23-1]+1, t2[k23]+1]
            cost22 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t2[k22+1]+1] + T[t2[k23-1]+1, t1[k12]+1] + c1 + T[t1[k11]+1, t2[k23]+1] - T[t2[k23-1]+1, t2[k23]+1]
        end
        k11 -= 1
        k12 -= 1
        k21 -= 1
        k22 -= 1
        k13 -= 1
        k23 -= 1
        alpha1 = copy(tour1[k11:k12])
        alpha2 = copy(tour2[k21:k22])
        tt1 = copy(tour1)
        
        deleteat!(tt1, [i for i=k11:k12])
        subtracted = false
        if k13 > k12
            k13 = k13 - (k12-k11+1)
            subtracted = true
        end
        for i=1:k22-k21+1
            insert!(tt1, i+k13-1, alpha2[i])
        end
        if round(cost11, digits =6) != round(find_tour_length(tt1, T), digits=6)
            println("fuck1   ", k13, "  ", k23, "  ", subtracted, "  ", k12-k11+1)
        end
        if subtracted 
            k13 += k12-k11+1
        end

        ttt1 = copy(tour1)
        alpha1 = copy(tour1[k11:k12])
        alpha2 = copy(tour2[k21:k22])
        deleteat!(ttt1, [i for i=k11:k12])
        if k13 > k12
            k13 = k13 - (k12-k11+1)
        end
        for i=1:k22-k21+1
            insert!(ttt1, i+k13-1, reverse(alpha2)[i])
        end
        if round(cost12, digits =6) != round(find_tour_length(ttt1, T), digits=6)
            println("fuck2")
        end
        alpha1 = copy(tour1[k11:k12])
        alpha2 = copy(tour2[k21:k22])
        tt2 = copy(tour2)
        deleteat!(tt2, [i for i=k21:k22])
        if k23 > k22
            k23 = k23 - (k22-k21+1)
        end
        for i=1:k12-k11+1
            insert!(tt2, i+k23-1, alpha1[i])
        end
        if round(cost21, digits =6) != round(find_tour_length(tt2, T), digits=6)
            println("fuck3")
        end
        alpha1 = copy(tour1[k11:k12])
        alpha2 = copy(tour2[k21:k22])
        ttt2 = copy(tour2)
        deleteat!(ttt2, [i for i=k21:k22])
        if k23 > k22
            k23 = k23 - (k22-k21+1)
        end
        for i=1:k12-k11+1
            insert!(ttt2, i+k23-1, reverse(alpha1)[i])
        end
        if round(cost22, digits =6) != round(find_tour_length(ttt2, T), digits=6)
            println("fuck4")
        end
    end
end
# print("cost2 - T[$(t2[k21-1]+1) , $(t2[k21]+1)] - c2 - T[$(t2[k22]+1), $(t2[k22+1]+1)] + T[$(t2[k21-1]+1), $(t2[k22+1]+1)] + T[$(t2[k23-1]+1), $(t1[k11]+1)] + c1 + T[$(t1[k12]+1), $(t2[k23]+1)] - T[$(t2[k23-1]+1), $(t2[k23]+1)]")

fuck3
fuck4
fuck1   4  10  false  4
fuck2
fuck3
fuck4
fuck1   4  11  false  4
fuck2
fuck3
fuck4
fuck1   4  12  false  4
fuck2
fuck3
fuck4
fuck1   4  13  false  4
fuck2
fuck3
fuck4
fuck1   4  14  false  4
fuck2
fuck3
fuck4
fuck1   4  15  false  4
fuck2
fuck1   4  16  false  4
fuck2
fuck1   4  17  false  4
fuck2
fuck1   4  18  false  4
fuck2
fuck1   4  19  false  4
fuck2
fuck1   4  20  false  4
fuck2
fuck1   4  21  false  4
fuck2
fuck4
fuck3
fuck4
fuck1   5  10  false  4
fuck2
fuck3
fuck4
fuck1   5  11  false  4
fuck2
fuck3
fuck4
fuck1   5  12  false  4
fuck2
fuck3
fuck4
fuck1   5  13  false  4
fuck2
fuck3
fuck4
fuck1   5  14  false  4
fuck2
fuck3
fuck4
fuck1   5  15  false  4
fuck2
fuck1   5  16  false  4
fuck2
fuck1   5  17  false  4
fuck2
fuck1   5  18  false  4
fuck2
fuck1   5  19  false  4
fuck2
fuck1   5  20  false  4
fuck2
fuck1   5  21  false  4
fuck2
fuck4
fuck3
fuck4
fuck1   6  10  false  4
fuck2
fuck3
fuck4
fuck1   6  11  false  4
fuck2
fuck3
fuck4
fuck1   6  12  false  4
fuck

In [232]:
for num=1:10000
    k11 = 3
    k12 = 6
    k21 = 7
    k22 = 13

    k13 = rand(8:length(tour1)-6)
    k23 = rand(15:length(tour2)-3)
    tour1 = copy(chrm.tours[1].Sequence)
    tour2 = copy(chrm.tours[2].Sequence);
    cost1 = chrm.tours[1].cost
    cost2 = chrm.tours[2].cost
    n_nodes = n

    c1 = sum(T[tour1[i]+1, tour1[i+1]+1] for i=k11:k12-1)
    c2 = sum(T[tour2[i]+1, tour2[i+1]+1] for i=k21:k22-1)
    t1 = copy(tour1)
    t2 = copy(tour2)
    pushfirst!(t1, 0)
    push!(t1, n_nodes+1)
    pushfirst!(t2, 0)
    push!(t2, n_nodes+1)
    k11 += 1
    k12 += 1
    k21 += 1
    k22 += 1
    k13 += 1
    k23 += 1
    if k13 == k11
        cost11 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t2[k21]+1] + c2 + T[t2[k22]+1, t1[k12+1]+1]
        cost12 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t2[k22]+1] + c2 + T[t2[k21]+1, t1[k12+1]+1]
    else
        cost11 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t1[k12+1]+1] + T[t1[k13-1]+1, t2[k21]+1] + c2 + T[t2[k22]+1, t1[k13]+1] - T[t1[k13-1]+1, t1[k13]+1]
        cost12 = cost1 - T[t1[k11-1]+1, t1[k11]+1] - c1 - T[t1[k12]+1, t1[k12+1]+1] + T[t1[k11-1]+1, t1[k12+1]+1] + T[t1[k13-1]+1, t2[k22]+1] + c2 + T[t2[k21]+1, t1[k13]+1] - T[t1[k13-1]+1, t1[k13]+1]
    end

    if k23 == k21
        cost21 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t1[k11]+1] + c1 + T[t1[k12]+1, t2[k22+1]+1]
        cost22 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t1[k12]+1] + c1 + T[t1[k11]+1, t2[k22+1]+1]
    else
        cost21 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t2[k22+1]+1] + T[t2[k23-1]+1, t1[k11]+1] + c1 + T[t1[k12]+1, t2[k23]+1] - T[t2[k23-1]+1, t2[k23]+1]
        cost22 = cost2 - T[t2[k21-1]+1, t2[k21]+1] - c2 - T[t2[k22]+1, t2[k22+1]+1] + T[t2[k21-1]+1, t2[k22+1]+1] + T[t2[k23-1]+1, t1[k12]+1] + c1 + T[t1[k11]+1, t2[k23]+1] - T[t2[k23-1]+1, t2[k23]+1]
    end
    k11 -= 1
    k12 -= 1
    k21 -= 1
    k22 -= 1
    k13 -= 1
    k23 -= 1
    alpha1 = copy(tour1[k11:k12])
    alpha2 = copy(tour2[k21:k22])
    tt1 = copy(tour1)

    deleteat!(tt1, [i for i=k11:k12])
    subtracted = false
    if k13 > k12
        k13 = k13 - (k12-k11+1)
        subtracted = true
    end
    for i=1:k22-k21+1
        insert!(tt1, i+k13-1, alpha2[i])
    end
    if round(cost11, digits =6) != round(find_tour_length(tt1, T), digits=6)
        println("fuck1   ", k13, "  ", k23)
    end
    if subtracted 
        k13 = k13 + k12-k11+1
    end

    ttt1 = copy(tour1)
    alpha1 = copy(tour1[k11:k12])
    alpha2 = copy(tour2[k21:k22])
    deleteat!(ttt1, [i for i=k11:k12])
    if k13 > k12
        k13 = k13 - (k12-k11+1)
    end
    for i=1:k22-k21+1
        insert!(ttt1, i+k13-1, reverse(alpha2)[i])
    end
    if round(cost12, digits =6) != round(find_tour_length(ttt1, T), digits=6)
        println("fuck2")
    end
    alpha1 = copy(tour1[k11:k12])
    alpha2 = copy(tour2[k21:k22])
    tt2 = copy(tour2)
    deleteat!(tt2, [i for i=k21:k22])
    subtracted = false
    if k23 > k22
        k23 = k23 - (k22-k21+1)
        subtracted = true
    end
    for i=1:k12-k11+1
        insert!(tt2, i+k23-1, alpha1[i])
    end
    if round(cost21, digits =6) != round(find_tour_length(tt2, T), digits=6)
        println("fuck3")
    end
    if subtracted 
        k23 = k23 + (k22-k21+1)
    end
    alpha1 = copy(tour1[k11:k12])
    alpha2 = copy(tour2[k21:k22])
    ttt2 = copy(tour2)
    deleteat!(ttt2, [i for i=k21:k22])
    if k23 > k22
        k23 = k23 - (k22-k21+1)
    end
    for i=1:k12-k11+1
        insert!(ttt2, i+k23-1, reverse(alpha1)[i])
    end
    if round(cost22, digits =6) != round(find_tour_length(ttt2, T), digits=6)
        println("fuck4")
    end
end

In [102]:
cost1 - T[7, 25] - c1 - T[2, 35] + T[7, 35] + T[19, 16] + c2 + T[41, 28] - T[19, 28]

277.1355402822403

In [154]:
print("cost1 - T[$(t1[k11-1]), $(t1[k11])] - c1 - T[$(t1[k12]), $(t1[k12+1])] + T[$(t1[k11-1]), $(t1[k12+1])] + T[$(t1[k13-1]), $(t2[k21])] + c2 + T[$(t2[k22]), $(t1[k13])] - T[$(t1[k13-1]), $(t1[k13])]")

cost1 - T[21, 7] - c1 - T[27, 2] + T[21, 2] + T[30, 16] + c2 + T[43, 27] - T[30, 27]

In [48]:
c2 + T[20,17] + T[42, 29] - T[20, 29]

140.6488552262334

In [49]:
229.48 - 12.39 + 140

357.09

In [281]:
include("costs.jl")
include("cross_escape.jl")

N_shift1_extensive (generic function with 1 method)

In [270]:
N_cross_super_extensive(chrm, T, n_nodes, 20)

Chromosome([63, 21, 46, 47, 28, 44, 26, 12, 53, 18  …  36, 19, 69, 59, 70, 68, 35, 20, 60, 27], 160.39395389777115, 131.43358521925742, Tour[Tour([63, 21, 46, 47, 28, 44, 26, 12, 53, 18, 7, 34, 66, 75, 74], 160.39395389777115), Tour([62, 9, 37, 64, 65, 10, 58, 13, 52, 6], 156.85269879834118), Tour([15, 2, 43, 31, 8, 30, 54, 24, 49, 17, 23, 48, 22, 55], 153.7555837970754), Tour([61, 73, 29, 1, 5, 16, 39, 38, 71, 57, 11, 25, 50, 32, 72, 42, 40, 41], 156.17224111884644), Tour([67, 3, 33, 45, 51, 56, 14, 4, 36, 19, 69, 59, 70, 68, 35, 20, 60, 27], 159.29460364497916)])

In [253]:
chrm = deepcopy(best_chrm)

Chromosome([26, 5, 47, 22, 6, 42, 23, 13, 24, 12  …  44, 14, 4, 37, 48, 8, 49, 15, 1, 21], 223.06096129062738, 189.95396364025012, Tour[Tour([26, 5, 47, 22, 6, 42, 23, 13, 24, 12  …  36, 16, 3, 17, 46, 11, 50, 45, 10, 31], 221.74352521764428), Tour([7, 25, 30, 27, 2, 35, 34, 19, 28, 20  …  44, 14, 4, 37, 48, 8, 49, 15, 1, 21], 223.06096129062738)])

In [254]:
N_cross_extensive(chrm, T, n_nodes, 20)

Chromosome([26, 5, 47, 22, 6, 42, 23, 13, 24, 12  …  44, 14, 4, 37, 48, 8, 49, 15, 1, 21], 223.06096129062738, 189.95396364025012, Tour[Tour([26, 5, 47, 22, 6, 42, 23, 13, 24, 12  …  36, 16, 3, 17, 46, 11, 50, 45, 10, 31], 221.74352521764428), Tour([7, 25, 30, 27, 2, 35, 34, 19, 28, 20  …  44, 14, 4, 37, 48, 8, 49, 15, 1, 21], 223.06096129062738)])

In [284]:
for ch in chrms
    t1 = time()
    chrm = deepcopy(ch)
    println(ch.fitness)
    N_cross_extensive(chrm, T, n_nodes, 5)
    println(chrm.fitness)
    println(time()-t1)
    println()
end

536.5764837930201
536.5764837930201
0.021126985549926758

533.7486011959656
533.7486011959656
0.030959129333496094

538.8993247800371
537.0147026021949
0.013238906860351562

527.0620728212473
526.5785101883381
0.0012841224670410156

539.5683286396599
539.5683286396599
0.02062392234802246

547.2744687524887
546.6377817812622
0.015126943588256836

550.3703546271661
550.0191461911403
0.00011110305786132812

545.0924551080885
545.0924551080885
0.015933990478515625

547.7759260667372
547.5942480706765
0.0068988800048828125

528.0879673761792
528.0879673761792
0.018581151962280273

